In [13]:
import os
import re
import pandas as pd
import importlib

## Extracción

In [14]:
import FuncionesLectura as fl
importlib.reload(fl)

df_corpus=fl.leerCorpus(hacerLimpieza=True)
print(df_corpus.shape)
df_corpus.sample(5)

Antes de limpiar: (6471, 9)
Después de eliminar español: (6102, 9)
Después de eliminar vacíos: (6101, 9)
Después de limpiar duplicados: (5623, 9)
(5621, 9)


d:\Repositorios\Tesis\O1_Corpus\FuncionesLectura.py:328: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df['pos'] = df['pos'].apply(lambda x: normalizePos(x))


,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
2948,JP-cuento.animal-2013_095,jose,wino,wino,wino,n.,macana,Macana.,JP-cuento.animal-2013.txt
1062,JC-vida.juanita-2013_076,Juanita,yoshinin pianka pianka koshkia ari ari ari ari,yoshinin pianka pianka k...,yoshin =n piank -a piank -a k...,n. =clit. v.tran -suf. v.tran -suf. v...,espíritu =ERG flechar -PERF flechar -PERF do...,"el espíritu le ha flechado, flechado, le ha he...",JC-vida.juanita-2013.txt
1634,JC.NC.IC-conversacion.varios-2013_210,Nelita,meno pari pain,meno pari pain,None,None,None,"""hubiera quemado primero""",JC.NC.IC-conversacion.varios-2013.txt
3354,JP-cuento.colorado-2013_325,José,haka parichakeska tapo aki tapo aki,haka parichakeska tapo aki ...,ha =ka paricha =keska tapo ak ...,dem. =clit. n. =suf. n. v.tra...,ese(SK) =NAR parrilla(CAST) =COMP parrilla ...,preparaba la parrilla,JP-cuento.colorado-2013.txt
5336,DICCIONARIOISKONAWA7_580,Dictionary,imi poroporo iki,None,None,intr.,None,derramar sangre,DICCIONARIOISKONAWA7.txt


In [15]:
#Se separa el corpus en dos partes, uno normal y otro del diccionario

df_diccionario=df_corpus[df_corpus["speaker"]=="Dictionary"]
df_corpus=df_corpus[df_corpus["speaker"]!="Dictionary"]

print(df_corpus.shape)
print(df_diccionario.shape)

df_corpus.to_json('corpus.json', orient='records', lines=True)
df_diccionario.to_json('diccionario.json', orient='records', lines=True)

(3938, 9)
(1683, 9)


## Prueba de lectura

In [16]:
df_corpus_leido = pd.read_json('corpus.json', lines=True)
df_diccionario_leido = pd.read_json('diccionario.json', lines=True)
df_corpus_leido.sample(5)

,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
1150,JC.NC.IC-conversacion.varios-2013_066,Juanita,hape,hape,hape,onom.,sonido.árbol,(Sonido de los árboles).,JC.NC.IC-conversacion.varios-2013.txt
2804,JP-cuento.colorado-2013_379,José,habon habon ika iki habon,habon habon ika iki ...,habon habon ik -a ik -i ...,n. n. v.intr. -suf. v.intr. -su...,lagartija lagartija copula -NOMLZ copula -IM...,estaban como lagartijas,JP-cuento.colorado-2013.txt
1242,JC.NC.IC-conversacion.varios-2013_176,Nelita,paeahon ka non tsoabo kena paoni,paeahon ka non tsoabo kena paoni,None,None,None,haciendo bebida nosotros a quienes invitamos,JC.NC.IC-conversacion.varios-2013.txt
3759,PC-cuento.mono-2013.eaf_a60,Pablo,xena pi,xena pi,None,None,None,estaba comiendo suri,PC-cuento.mono-2013.eaf
3611,NC-diseños.iskonawa4-2018.eaf_a85,Nelita,wistewan irabo y narabe ainbo,None,None,None,None,un hombre y dos mujeres,NC-diseños.iskonawa4-2018.eaf


## Validaciones

In [12]:
#Verificar que las columnas id,	speaker,	transcription, free_translation, file no tengan valores nulos
assert df_corpus_leido.id.isnull().sum() == 0
assert df_corpus_leido.speaker.isnull().sum() == 0  
assert df_corpus_leido.transcription.isnull().sum() == 0
assert df_corpus_leido.free_translation.isnull().sum() == 0
assert df_corpus_leido.file.isnull().sum() == 0

print('No hay valores nulos en el corpus')

No hay valores nulos en el corpus
